# Cut Whole Slide Images into SpatialData objects

In [1]:
%load_ext autoreload
%autoreload 2

from datetime import datetime
from pathlib import Path

import pandas as pd
from loguru import logger

from plex_pipe.core_cutting import RoiPreparationController, LocalFileStrategy
from plex_pipe import load_config

## Read in config

In [ ]:
# load analysis configuration
config_path = r'../examples/example_pipeline_config.yaml'

config = load_config(config_path)

2026-02-18 17:28:12.627 | WARNING  | plex_pipe.processors.base:_log_missing_optional_params:49 - InstansegSegmenter.Params: Parameter overlap not provided. Using default value: 80.


## Define the logger

In [3]:
log_file = config.log_dir_path / f"rois_cutting_{datetime.now():%Y-%m-%d_%H-%M-%S}.log"

logger.remove()
l = logger.add(lambda msg: print(msg, end=""))
l = logger.add(log_file, level="DEBUG", enqueue=True)

## Define ROIs for processing

In [4]:
df_path = config.roi_info_file_path

df = pd.read_pickle(df_path)
df.head()

,roi_name,row_start,row_stop,column_start,column_stop,poly_type,polygon_vertices
0,ROI_000,256.0,5056.0,256.0,5120.0,rectangle,"[[5056.0, 5120.0], [5056.0, 256.0], [256.0, 25..."
1,ROI_001,128.0,4992.0,6912.0,11776.0,rectangle,"[[4992.0, 11776.0], [4992.0, 6912.0], [128.0, ..."


In [5]:
# # create a subset of rois (optional)
# df = df[:1]
# df

## Discover and choose marker files based on config

In [6]:
strategy = LocalFileStrategy(config = config)

2026-02-18 17:28:20.379 | INFO     | plex_pipe.core_cutting.channel_scanner:scan_channels_from_list:73 - Discovered 4 channels:
2026-02-18 17:28:20.379 | INFO     | plex_pipe.core_cutting.channel_scanner:scan_channels_from_list:75 - 001_CD45 <- ..\examples\input\sample_1.0.4_R000_Cy5_CD45-AF647_FINAL_AFR_F.tiff
2026-02-18 17:28:20.379 | INFO     | plex_pipe.core_cutting.channel_scanner:scan_channels_from_list:75 - 001_DAPI <- ..\examples\input\sample_1.0.4_R000_DAPI__FINAL_F.tiff
2026-02-18 17:28:20.380 | INFO     | plex_pipe.core_cutting.channel_scanner:scan_channels_from_list:75 - 001_NaKATPase <- ..\examples\input\sample_1.0.4_R000_Cy7_NaKATPase-AF750_FINAL_AFR_F.tiff
2026-02-18 17:28:20.380 | INFO     | plex_pipe.core_cutting.channel_scanner:scan_channels_from_list:75 - 001_bCat <- ..\examples\input\sample_1.0.4_R000_Cy3_bCat-AF555_FINAL_AFR_F.tiff
2026-02-18 17:28:20.380 | INFO     | plex_pipe.core_cutting.channel_scanner:scan_channels_from_list:125 - Ignoring markers = ['bCat']
2

In [7]:
strategy.channel_map

{'CD45': '..\\examples\\input\\sample_1.0.4_R000_Cy5_CD45-AF647_FINAL_AFR_F.tiff',
 'DAPI': '..\\examples\\input\\sample_1.0.4_R000_DAPI__FINAL_F.tiff',
 'NaKATPase': '..\\examples\\input\\sample_1.0.4_R000_Cy7_NaKATPase-AF750_FINAL_AFR_F.tiff'}

In [8]:
# # for tests you can request a smaller set of channels here
# selected_keys = ["DAPI"]
# strategy.channel_map = {k: strategy.channel_map[k] for k in selected_keys if k in strategy.channel_map}
# strategy.channel_map

## Run ROI cutting

In [ ]:
controller = RoiPreparationController(
    metadata_df = df, # df defines which cores to process
    file_strategy = strategy, # defines which channels to use
    temp_dir = config.roi_dir_tif_path,
    output_dir = config.roi_dir_output_path,
    margin = config.roi_cutting.margin,
    mask_value = config.roi_cutting.mask_value,
    max_pyramid_levels = config.sdata_storage.max_pyramid_level,
    chunk_size = config.sdata_storage.chunk_size,
    downscale = config.sdata_storage.downscale,
    temp_roi_delete = False,
    )

controller.run()

2026-02-18 17:28:24.689 | INFO     | plex_pipe.core_cutting.controller:run:98 - Starting ROI preparation controller...
2026-02-18 17:28:24.690 | INFO     | plex_pipe.core_cutting.input_strategy:yield_ready_channels:191 - Local file for CD45 verified: ..\examples\input\sample_1.0.4_R000_Cy5_CD45-AF647_FINAL_AFR_F.tiff
2026-02-18 17:28:24.690 | INFO     | plex_pipe.core_cutting.controller:run:103 - Channel CD45 ready. Starting cutting...
2026-02-18 17:28:25.187 | DEBUG    | plex_pipe.core_cutting.controller:cut_channel:84 - Cut and saved ROI ROI_000, channel CD45.
2026-02-18 17:28:25.306 | DEBUG    | plex_pipe.core_cutting.controller:cut_channel:84 - Cut and saved ROI ROI_001, channel CD45.
2026-02-18 17:28:25.307 | DEBUG    | plex_pipe.core_cutting.controller:cut_channel:89 - Closed file handle for channel CD45.
2026-02-18 17:28:25.308 | INFO     | plex_pipe.core_cutting.input_strategy:yield_ready_channels:191 - Local file for DAPI verified: ..\examples\input\sample_1.0.4_R000_DAPI__FIN

d:\plex-pipe\.venv\Lib\site-packages\ome_zarr\writer.py:301: FutureWarning: Passing storage-related arguments via **kwargs is deprecated. Please use the 'zarr_store_kwargs' parameter instead. **kwargs will be removed in a future version.
  da_delayed = da.to_zarr(
d:\plex-pipe\.venv\Lib\site-packages\ome_zarr\writer.py:301: FutureWarning: Passing storage-related arguments via **kwargs is deprecated. Please use the 'zarr_store_kwargs' parameter instead. **kwargs will be removed in a future version.
  da_delayed = da.to_zarr(
d:\plex-pipe\.venv\Lib\site-packages\ome_zarr\writer.py:301: FutureWarning: Passing storage-related arguments via **kwargs is deprecated. Please use the 'zarr_store_kwargs' parameter instead. **kwargs will be removed in a future version.
  da_delayed = da.to_zarr(


2026-02-18 17:28:28.823 | INFO     | plex_pipe.core_cutting.assembler:assemble_core:112 - ROI 'ROI_000' assembled with channels: ['CD45', 'DAPI', 'NaKATPase']


d:\plex-pipe\.venv\Lib\site-packages\ome_zarr\writer.py:301: FutureWarning: Passing storage-related arguments via **kwargs is deprecated. Please use the 'zarr_store_kwargs' parameter instead. **kwargs will be removed in a future version.
  da_delayed = da.to_zarr(
d:\plex-pipe\.venv\Lib\site-packages\ome_zarr\writer.py:301: FutureWarning: Passing storage-related arguments via **kwargs is deprecated. Please use the 'zarr_store_kwargs' parameter instead. **kwargs will be removed in a future version.
  da_delayed = da.to_zarr(
d:\plex-pipe\.venv\Lib\site-packages\ome_zarr\writer.py:301: FutureWarning: Passing storage-related arguments via **kwargs is deprecated. Please use the 'zarr_store_kwargs' parameter instead. **kwargs will be removed in a future version.
  da_delayed = da.to_zarr(


2026-02-18 17:28:31.552 | INFO     | plex_pipe.core_cutting.assembler:assemble_core:112 - ROI 'ROI_001' assembled with channels: ['CD45', 'DAPI', 'NaKATPase']
2026-02-18 17:28:31.554 | INFO     | plex_pipe.core_cutting.controller:run:117 - All cores assembled. Controller run complete.
